
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


In [3]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
from misc_functions import calculate_RTs


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np
import copy


from misc_functions import calculate_RTs, save_sig_chans, save_channels_to_file
import pickle

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def relabel_axes(old_min, old_max, new_min, new_max):
    scale = (new_max - new_min) / (old_max - old_min)

    def format_func(value, tick_number):
        return new_min + value * scale

    plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))

### make it into a function

In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
from misc_functions import calculate_RTs


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np
import copy

from misc_functions import calculate_RTs, save_sig_chans, save_channels_to_file
import pickle


def get_baseline(inst: mne.io.BaseRaw, base_times: tuple[float, float]):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(inst, "experimentStart", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")
    del inst
    return HG_base


def plot_HG_and_stats(sub, task, output_name, event=None, times=(-1, 1.5), base_times=(-0.5, 0), LAB_root=None, channels=None, full_trial_base=False):
    """
    Plot high gamma (HG) and statistics for a given subject and task using specified event.
    
    Parameters:
    - sub (str): The subject identifier.
    - task (str): The task identifier.
    - output_name (str): The name for the output files.
    - event (str, optional): Event name to process. Defaults to None.
    - times (tuple, optional): A tuple indicating the start and end times for processing. Defaults to (-1, 1.5).
    - base_times (tuple, optional): A tuple indicating the start and end base times for processing. Defaults to (-0.5, 0).
    - LAB_root (str, optional): The root directory for the lab. Will be determined based on OS if not provided. Defaults to None.
    - channels (list of strings, optional): The channels to plot and get stats for. Default is all channels.
    - full_trial_base (boolean): Whether to use the full trial as the baseline period. Default is False.
    This function will process the provided event for a given subject and task. 
    High gamma (HG) will be computed, and statistics will be calculated and plotted. 
    The results will be saved to output files.
    """
    pass

sub='D0057'
task='GlobalLocal'
output_name="stimulus_RMMF1-5_experimentStartBase1secTo101Sec"
event="Stimulus"
times=(-1, 1.5)
base_times=(1, 101)
LAB_root=None
channels=None
full_trial_base=False

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt': # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else: # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data(task, root=LAB_root)
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub, extension='.edf', desc='clean', preload=False)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

try:
    good = crop_empty_data(filt)
except ValueError as ve:
    print(f"Error encountered during cropping: {ve}")

print(f"good channels before dropping bads: {len(good.ch_names)}")
print(f"filt channels before dropping bads: {len(filt.ch_names)}")

good.info['bads'] = channel_outlier_marker(good, 3, 2)
print("Bad channels in 'good':", good.info['bads'])

filt.drop_channels(good.info['bads']) #this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
good.drop_channels(good.info['bads'])

print("Bad channels in 'good' after dropping once:", good.info['bads'])

print(f"good channels after dropping bads: {len(good.ch_names)}")
print(f"filt channels after dropping bads: {len(filt.ch_names)}")


HG_base = get_baseline(filt, base_times)
good.load_data()


# If channels is None, use all channels
if channels is None:
    channels = good.ch_names
else:
    # Validate the provided channels
    invalid_channels = [ch for ch in channels if ch not in good.ch_names]
    if invalid_channels:
        raise ValueError(f"The following channels are not valid: {invalid_channels}")

    # Use only the specified channels
    good.pick_channels(channels)

ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Create a baseline EpochsTFR using the stimulus event

# adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
# trials = trial_ieeg(good, "Stimulus", adjusted_base_times, preload=True)
# outliers_to_nan(trials, outliers=10)
# HG_base = gamma.extract(trials, copy=False, n_jobs=1)
# crop_pad(HG_base, "0.5s")

# adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]

# Epoching and HG extraction for the specified event
times_adj = [times[0] - 0.5, times[1] + 0.5]
trials = trial_ieeg(good, event, times_adj, preload=True, reject_by_annotation=False)
outliers_to_nan(trials, outliers=10)
HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_ev1, "0.5s")
HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='zscore')

HG_base.decimate(2)
HG_ev1.decimate(2)
RTs, skipped = calculate_RTs(good)
avg_RT = np.median(RTs)

HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))


if np.isnan(HG_ev1_evoke_rescaled.data).any():
    print("Data contains NaN values.")
    # HG_ev1_evoke_rescaled.data = np.nan_to_num(HG_ev1_evoke_rescaled.data)

# Debugging: Print shapes of data arrays
print(f"Shape of HG_base._data: {HG_base._data.shape}")
print(f"Shape of HG_ev1._data: {HG_ev1._data.shape}")
print(f"shape of HG_ev1_evoke_rescaled._data: {HG_ev1_evoke_rescaled._data.shape}")
# For stimulus event, plot average RT
if event == "Stimulus":
    HG_ev1_evoke_rescaled_noNaN = copy.deepcopy(HG_ev1_evoke_rescaled)
    HG_ev1_evoke_rescaled_noNaN.data = np.nan_to_num(HG_ev1_evoke_rescaled_noNaN.data)

    fig = HG_ev1_evoke_rescaled_noNaN.plot(unit=False, scalings=dict(sEEG=1))
    for ax in fig.axes:
        ax.axvline(x=avg_RT, color='r', linestyle='--')
    fig.savefig(save_dir + '_HG_ev1_Stimulus_zscore.png')
else:
    fig = HG_ev1_evoke_rescaled_noNaN.plot(unit=False, scalings=dict(sEEG=1))
    fig.savefig(save_dir + f'_HG_ev1_{output_name}_zscore.png')

# Calculate statistics and define channel names
mask = stats.time_perm_cluster(HG_ev1._data, HG_base._data, 0.05, axis=0, n_perm=1000, n_jobs=6, ignore_adjacency=1)

channels = good.ch_names

#save channels with their indices 
save_channels_to_file(channels, sub, task, save_dir)

# save significant channels to a json
save_sig_chans(f'{output_name}', mask, channels, sub, save_dir)

# plot stats
plt.figure(figsize=(16, 8))
plt.imshow(mask, aspect='auto')
relabel_axes(0, 2500, -1000, 1500)
plt.savefig(save_dir + f'_{output_name}_stats_big.png', dpi=300)

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Applying baseline correction (mode: zscore)


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\calc\scaling.py:87: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(basedata, axis=axis, keepdims=True)
c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\jz421\AppData\Local\Temp\ipykernel_14324\606351779.py:144: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_14324\606351779.py:145: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=2 parameter will result in a sampling frequency of 1024.0 Hz, which can cause aliasing artifacts.
  HG_ev1.decimate(2)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_14324\606351779.py:150: RuntimeWarning: Mean of empty slice
  HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))


Data contains NaN values.
Shape of HG_base._data: (1, 175, 102401)
Shape of HG_ev1._data: (448, 175, 2561)
shape of HG_ev1_evoke_rescaled._data: (175, 5121)


In [26]:
# print(HG_ev1._data)

# if np.isnan(HG_ev1._data).any():
#     print("Data contains NaN values.")
data = HG_ev1.get_data()
nan_channels = {}
for epoch_num in range(data.shape[0]):
    nan_channels_epoch = []
    for i, channel_data in enumerate(data[epoch_num, :, :]):
        if np.isnan(channel_data).any():
            nan_channels_epoch.append(HG_ev1.info['ch_names'][i])
            
    if nan_channels_epoch:  # Only add to the dictionary if the list is not empty
        nan_channels[f'Epoch {epoch_num + 1}'] = nan_channels_epoch
        print(f"Channels with NaN values in Epoch {epoch_num + 1}: {nan_channels_epoch}")




Channels with NaN values in Epoch 35: ['RMMF14']
Channels with NaN values in Epoch 38: ['RMMF14']
Channels with NaN values in Epoch 39: ['RMMF14']
Channels with NaN values in Epoch 43: ['RPIP12']
Channels with NaN values in Epoch 44: ['RPIP12']
Channels with NaN values in Epoch 60: ['LAMT1']
Channels with NaN values in Epoch 66: ['RPI13', 'RPI14']
Channels with NaN values in Epoch 70: ['RMMF14']
Channels with NaN values in Epoch 77: ['RAIF10']
Channels with NaN values in Epoch 92: ['RMMT2', 'LAMT1']
Channels with NaN values in Epoch 99: ['RPIT10']
Channels with NaN values in Epoch 109: ['LMMT11', 'LMMT12']
Channels with NaN values in Epoch 134: ['RAST5', 'RAST6']
Channels with NaN values in Epoch 136: ['RPI13']
Channels with NaN values in Epoch 137: ['RPI13']
Channels with NaN values in Epoch 151: ['RPI13', 'RPIT5']
Channels with NaN values in Epoch 154: ['LMMT11', 'LMMT12']
Channels with NaN values in Epoch 155: ['LMMT11']
Channels with NaN values in Epoch 165: ['RPI13', 'LMMT9']
Chan

In [28]:
data = HG_base.get_data()
nan_channels = {}
for epoch_num in range(data.shape[0]):
    nan_channels_epoch = []
    for i, channel_data in enumerate(data[epoch_num, :, :]):
        if np.isnan(channel_data).any():
            nan_channels_epoch.append(HG_base.info['ch_names'][i])
            
    if nan_channels_epoch:  # Only add to the dictionary if the list is not empty
        nan_channels[f'Epoch {epoch_num + 1}'] = nan_channels_epoch
        print(f"Channels with NaN values in Epoch {epoch_num + 1}: {nan_channels_epoch}")



Channels with NaN values in Epoch 1: ['RAIF10', 'RAIT10', 'RMMT11', 'RPIT10', 'LMMT11']


In [33]:
# Get the data from the Raw object
data, times = good[:, :]

# Check each channel for NaNs
nan_channels = []
for i, channel_data in enumerate(data):
    if np.isnan(channel_data).any():
        nan_channels.append(filt.info['ch_names'][i])

print("Channels with NaN values:", nan_channels)



Channels with NaN values: []


In [ ]:
def get_baseline(inst: mne.io.BaseRaw, base_times: tuple[float, float]):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
    trials = trial_ieeg(inst, "experimentStart", adjusted_base_times, preload=True)
    outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")
    del inst
    return HG_base



sub='D0057'
task='GlobalLocal'
output_name="stimulus_RMMF1-5_preExperimentBaselineTest"
event="Stimulus"
times=(-1, 1.5)
base_times=(1,11)
LAB_root=None
channels=None
full_trial_base=False

if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt': # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else: # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data(task, root=LAB_root)
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub, extension='.edf', desc='clean', preload=False)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

try:
    good = crop_empty_data(filt)
except ValueError as ve:
    print(f"Error encountered during cropping: {ve}")

print(f"good channels before dropping bads: {len(good.ch_names)}")
print(f"filt channels before dropping bads: {len(filt.ch_names)}")

good.info['bads'] = channel_outlier_marker(good, 3, 2)
print("Bad channels in 'good':", good.info['bads'])

filt.drop_channels(good.info['bads']) #this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
good.drop_channels(good.info['bads'])

print("Bad channels in 'good' after dropping once:", good.info['bads'])

print(f"good channels after dropping bads: {len(good.ch_names)}")
print(f"filt channels after dropping bads: {len(filt.ch_names)}")


HG_base = get_baseline(filt, base_times)

inst = filt
inst = inst.copy()
inst.load_data()
ch_type = inst.get_channel_types(only_data_chs=True)[0]
inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
trials = trial_ieeg(inst, "experimentStart", adjusted_base_times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

In [5]:
inst = filt
base_times = (1,100)
inst = inst.copy()
test = inst.load_data()
ch_type = inst.get_channel_types(only_data_chs=True)[0]
inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

adjusted_base_times = [base_times[0] - 0.5, base_times[1] + 0.5]
trials = trial_ieeg(inst, "experimentStart", adjusted_base_times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=1)
crop_pad(HG_base, "0.5s")

NameError: name 'filt' is not defined

In [56]:
HG_base.plot()

In [38]:
raw = filt
events, ids = mne.events_from_annotations(raw)
event = "experimentStart"
dat_ids = [ids[i] for i in mne.event.match_event_names(ids, event)]
rev = {k: v for k, v in ids.items() if v in dat_ids}

Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75', 'experimentStart']


In [39]:
print("Event IDs:", ids)
print("First few events:", events[:10])
print("dat_ids: ", dat_ids)
print("rev: ", rev)

Event IDs: {'Response/c25/n25': 1, 'Response/c25/r25': 2, 'Response/c25/r75': 3, 'Response/c25/s25': 4, 'Response/c25/s75': 5, 'Response/c75/r25': 6, 'Response/c75/r75': 7, 'Response/c75/s25': 8, 'Response/c75/s75': 9, 'Response/i25/r25': 10, 'Response/i25/r75': 11, 'Response/i25/s25': 12, 'Response/i25/s75': 13, 'Response/i75/n25': 14, 'Response/i75/n75': 15, 'Response/i75/r25': 16, 'Response/i75/r75': 17, 'Response/i75/s25': 18, 'Response/i75/s75': 19, 'Stimulus/c25/n25': 20, 'Stimulus/c25/r25': 21, 'Stimulus/c25/r75': 22, 'Stimulus/c25/s25': 23, 'Stimulus/c25/s75': 24, 'Stimulus/c75/r25': 25, 'Stimulus/c75/r75': 26, 'Stimulus/c75/s25': 27, 'Stimulus/c75/s75': 28, 'Stimulus/i25/n75': 29, 'Stimulus/i25/r25': 30, 'Stimulus/i25/r75': 31, 'Stimulus/i25/s25': 32, 'Stimulus/i25/s75': 33, 'Stimulus/i75/n25': 34, 'Stimulus/i75/n75': 35, 'Stimulus/i75/r25': 36, 'Stimulus/i75/r75': 37, 'Stimulus/i75/s25': 38, 'Stimulus/i75/s75': 39, 'experimentStart': 40}
First few events: [[      0       0   

In [17]:
HG_base_data = HG_base.get_data()

print(HG_base_data.shape)

events, event_id = mne.events_from_annotations(filt)
print('Found %s events, first five:' % len(events))
print(events[:5])

(0, 175, 20481)
Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75', 'experimentStart']
Found 2 events, first five:
(array([[      0,       0,      40],
       [2784657,       0,      29],
       [2793394,       0,      24],
       ...,
    

In [ ]:
test = plot_HG_and_stats(sub='D0057', task="GlobalLocal", output_name="stimulus_RMMF1-5_experimentStartBase10SECONDTEST", event="Stimulus", times=(-1, 1.5), base_times=(0,10), channels=["RMMF1", "RMMF2", "RMMF3", "RMMF4", "RMMF5"])

In [20]:
[a for a in filt.annotations if a['description'] == 'experimentStart']

[OrderedDict([('onset', 0.0),
              ('duration', 0.0),
              ('description', 'experimentStart'),
              ('orig_time',
               datetime.datetime(2023, 7, 3, 12, 31, 29, tzinfo=datetime.timezone.utc))])]

In [20]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")

layout = get_data('GlobalLocal', root=LAB_root)
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject='D0057', extension='.edf', desc='clean', preload=True)
trial_ieeg(filt, "experimentStart", (0, 101), preload=True)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Reading 0 ... 8243199  =      0.000 ...  4025.000 secs...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimulus/i75/r25', 'Stimulus/i75/r75', 'Stimulus/i75/s25', 'Stimulus/i75/s75', 'experimentStart']
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2 events and 206849 original t

Number of events,2
Events,experimentStart: 2
Time range,0.000 – 101.000 s
Baseline,off


In [ ]:
plot_HG_and_stats(sub='D0065', task="GlobalLocal", output_name="incongruent", event="/i", times=(-1, 1.5))

In [ ]:
plot_HG_and_stats(sub='D0065', task="GlobalLocal", output_name="switch", event="/s", times=(-1, 1.5))

In [ ]:
plot_HG_and_stats(sub='D0065', task="GlobalLocal", output_name="repeat", event="/r", times=(-1, 1.5))

# clean up this damn notebook soon 9/10

### trying to make the pre-experiment baseline of five minutes right after recording starts

In [19]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

sub='D0057'
layout = get_data("GlobalLocal", root=LAB_root)
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub, extension='.edf', desc='clean', preload=False)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
good = crop_empty_data(filt)
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
good.load_data()



channels = good.ch_names


ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)


times=[0,100]

trials = trial_ieeg(good, "experimentStart", times, preload=True)
outliers_to_nan(trials, outliers=10)
HG_base = gamma.extract(trials, copy=False, n_jobs=-1)
crop_pad(HG_base, "0.5s")

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 6024958  =      0.000 ...  2941.874 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['Response/c25/n25', 'Response/c25/r25', 'Response/c25/r75', 'Response/c25/s25', 'Response/c25/s75', 'Response/c75/r25', 'Response/c75/r75', 'Response/c75/s25', 'Response/c75/s75', 'Response/i25/r25', 'Response/i25/r75', 'Response/i25/s25', 'Response/i25/s75', 'Response/i75/n25', 'Response/i75/n75', 'Response/i75/r25', 'Response/i75/r75', 'Response/i75/s25', 'Response/i75/s75', 'Stimulus/c25/n25', 'Stimulus/c25/r25', 'Stimulus/c25/r75', 'Stimulus/c25/s25', 'Stimulus/c25/s75', 'Stimulus/c75/r25', 'Stimulus/c75/r75', 'Stimulus/c75/s25', 'Stimulus/c75/s75', 'Stimulus/i25/n75', 'Stimulus/i25/r25', 'Stimulus/i25/r75', 'Stimulus/i25/s25', 'Stimulus/i25/s75', 'Stimulus/i75/n25', 'Stimulus/i75/n75', 'Stimul

100%|██████████| 2/2 [02:08<00:00, 64.30s/it]


Number of events,2
Events,experimentStart: 2
Time range,0.500 – 99.500 s
Baseline,off


In [20]:
fig = HG_base.plot()


In [21]:
HG_base._data

array([[[3.17678905e-06, 3.19554465e-06, 3.21445668e-06, ...,
         2.94870779e-06, 2.93768426e-06, 2.92631830e-06],
        [4.41296288e-06, 4.43006229e-06, 4.44649686e-06, ...,
         3.68066458e-06, 3.67953771e-06, 3.67840130e-06],
        [3.88735043e-06, 3.87360615e-06, 3.85933163e-06, ...,
         3.26325585e-06, 3.26373492e-06, 3.26433019e-06],
        ...,
        [3.71415581e-06, 3.69500162e-06, 3.67553389e-06, ...,
         5.58981583e-06, 5.58533611e-06, 5.58016518e-06],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [4.19572007e-06, 4.19072057e-06, 4.18549143e-06, ...,
         9.24691085e-06, 9.25795393e-06, 9.26732901e-06]],

       [[3.86658758e-06, 3.88489616e-06, 3.90320838e-06, ...,
         3.52051325e-06, 3.52580901e-06, 3.53100768e-06],
        [2.76941114e-06, 2.78063249e-06, 2.79206870e-06, ...,
         3.21512357e-06, 3.21370499e-06, 3.21325706e-06],
        [2.46078025e-06, 